In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

2024-12-27 14:48:33.199718: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.17.0'

In [3]:
!python --version

Python 3.10.12


In [4]:
from keras.preprocessing.image import load_img
from keras.applications.xception import preprocess_input
import PIL

In [5]:
classes = ['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

classes

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [33]:
model = keras.models.load_model('./model_vf_24_0.958.h5.keras')

In [34]:
def get_prediction(img_url, classes):
    img = load_img(img_url, target_size=(150, 150))
    x = np.array(img)
    X = np.array([x])
    X = preprocess_input(X)
    preds = model.predict(X)
    return dict(zip(classes, preds[0]))

    

In [40]:
img_url = ('./Residential_1000.jpg')
get_prediction(img_url, classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step


{'AnnualCrop': 2.912262e-10,
 'Forest': 1.7284238e-10,
 'HerbaceousVegetation': 2.5663874e-06,
 'Highway': 0.0033005045,
 'Industrial': 0.45216134,
 'Pasture': 7.4172235e-10,
 'PermanentCrop': 2.1298106e-06,
 'Residential': 0.54453343,
 'River': 6.8858214e-10,
 'SeaLake': 6.1157026e-12}

In [42]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('terrain-classification.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpu3t6bzo5/assets


INFO:tensorflow:Assets written to: /tmp/tmpu3t6bzo5/assets


Saved artifact at '/tmp/tmpu3t6bzo5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  140468723138800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140468723144608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140469327781680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140469323134432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140470823012832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140469322939232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140469322768176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140468725107344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140468725109104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140468724717120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14046872471571

W0000 00:00:1735335592.656211   41780 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1735335592.661618   41780 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-12-27 17:39:52.673223: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpu3t6bzo5
2024-12-27 17:39:52.675238: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-27 17:39:52.675291: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpu3t6bzo5
2024-12-27 17:39:52.710301: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-12-27 17:39:52.715308: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-27 17:39:53.094602: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpu3t6bzo5
2024-12-27 17:39:53.125115: I tensorflow/cc/saved_model/loader.cc

In [43]:
import tensorflow.lite as tflite

In [47]:
interpreter = tflite.Interpreter(model_path='terrain-classification.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [48]:
img_url = './Residential_1000.jpg'
img = load_img(img_url, target_size=(150, 150))
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

In [50]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
dict(zip(classes, preds[0]))

{'AnnualCrop': 2.912257e-10,
 'Forest': 1.72842e-10,
 'HerbaceousVegetation': 2.5663796e-06,
 'Highway': 0.003300501,
 'Industrial': 0.45216253,
 'Pasture': 7.4171896e-10,
 'PermanentCrop': 2.129804e-06,
 'Residential': 0.5445323,
 'River': 6.885813e-10,
 'SeaLake': 6.115678e-12}

In [51]:
from keras_image_helper import create_preprocessor

In [58]:
preprocessor = create_preprocessor('xception', target_size=(150, 150))
img_url = './AnnualCrop_1000.jpg'
X = preprocessor.from_path(img_url)
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
dict(zip(classes, preds[0]))

{'AnnualCrop': 0.99991465,
 'Forest': 4.3515373e-13,
 'HerbaceousVegetation': 7.984264e-05,
 'Highway': 3.5596227e-06,
 'Industrial': 2.1262835e-08,
 'Pasture': 2.4187857e-10,
 'PermanentCrop': 1.9598576e-06,
 'Residential': 1.8129019e-12,
 'River': 2.2560126e-08,
 'SeaLake': 1.1681824e-08}

# Tensorflow lite


In [59]:
!pip install tflite-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 4.3 MB/s eta 0:00:00a 0:00:01


In [5]:
import tflite_runtime.interpreter as tflite 
from keras_image_helper import create_preprocessor

In [10]:
classes = ['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

classes

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [14]:
preprocessor = create_preprocessor('xception', target_size=(150, 150))
img_url = './AnnualCrop_1004.jpg'
X = preprocessor.from_path(img_url)

In [15]:
# Load the TFLite model
interpreter = tflite.Interpreter(model_path="./terrain-classification.tflite") 
interpreter.allocate_tensors()

# Get input and output details
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

# Prepare input data
interpreter.set_tensor(input_index, X)

# Run inference
interpreter.invoke()

# Get output data
preds = interpreter.get_tensor(output_index)
dict(zip(classes, preds[0]))

{'AnnualCrop': 0.99901104,
 'Forest': 6.1193684e-19,
 'HerbaceousVegetation': 1.38757645e-08,
 'Highway': 0.000988261,
 'Industrial': 1.2046471e-14,
 'Pasture': 3.6291554e-15,
 'PermanentCrop': 5.1203884e-08,
 'Residential': 3.6226745e-19,
 'River': 5.613105e-07,
 'SeaLake': 8.709841e-12}